In [1]:
from functools import reduce
from math import sqrt
from sklearn import linear_model
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy as sp

In [2]:
#(c)
def standardization(x, mean, std) : 
    return (x-mean)/std

def get_tot_diff_with_mean(x) : 
    mean_x = get_mean(x)
    return sum(map(lambda x : (x - mean_x) ** 2, x))

def get_mean(x) : 
    return (reduce(lambda value_x1, value_x2 : value_x1 + value_x2, x))/len(x)

def get_var(x) : 
    return get_tot_diff_with_mean(x)/len(x)

def get_std(x) : 
    return sqrt(get_tot_diff_with_mean(x)/(len(x) - 1))

def get_cov(x, y) : 
    mean_x = get_mean(x)
    mean_y = get_mean(y)
    sum_of_diff = sum(map(lambda value_x, value_y: (value_x-mean_x)*(value_y-mean_y), x, y))
    return sum_of_diff / (len(x) - 1)

def get_cor(x, y) : 
    return get_cov(x, y) / (get_std(x) * get_std(y))

def get_SSE(y, y_hat) :
    return sum(map(lambda value_y, value_y_hat : (value_y-value_y_hat) ** 2, y, y_hat)) / (len(y) - 2)

def get_slope(x, y) : 
    mean_x = get_mean(x)
    mean_y = get_mean(y)
    return sum(map(lambda value_x, value_y : (value_x - mean_x) * (value_y - mean_y), x, y)) / get_tot_diff_with_mean(x)

def get_intercept(x, y) : 
    return get_mean(y) - (get_slope(x, y) * get_mean(x))

def get_SST(y) : 
    return get_tot_diff_with_mean(y)

def get_R_sqr(y, y_hat) : 
    return 1 - (get_SSE(y, y_hat)/get_SST(y))

def get_std_err_of_intercept(y, y_hat, x) : 
    mean_x = get_mean(x)
    var = get_SSE(y, y_hat)
    std_err_of_intercept = sqrt(var) * sqrt((1/len(x)) + (mean_x ** 2 / get_tot_diff_with_mean(x)))    
    return std_err_of_intercept

def get_std_err_of_slope(y, y_hat, x) : 
    std_err_of_slope = sqrt(get_SSE(y, y_hat)) / sqrt(get_tot_diff_with_mean(x))
    return std_err_of_slope

def get_y_hat(slope, intercept, x) : 
    y_hat = [intercept + slope * x for x in x]
    return y_hat

def get_predicted_value(slope, intercept, given_x) : 
    predicted_value = intercept + slope * given_x
    return predicted_value

def get_std_err_of_predicted_value(SSE, x, given_x) : 
    mean_x = get_mean(x)
    std_err_of_predicted_value = SSE * sqrt(1 + (1/len(x)) + (((given_x - mean_x) ** 2) / get_tot_diff_with_mean(x)))
    return std_err_of_predicted_value

def get_confidence_interval_of_predicted_values(predicted_value, std_err_of_predicted_value, x, alpha=0.95) : 
    confidence_interval = sp.stats.t.interval(alpha=0.95, df=len(x)-2)
    print(std_err_of_predicted_value)
    print(predicted_value)
    print(confidence_interval)
    return (predicted_value + (std_err_of_predicted_value * confidence_interval[0]), predicted_value + (std_err_of_predicted_value * confidence_interval[1]))

#### 3.1
감독자 직무수행능력 데이터를 이용하여, 식 (3.12)에 주어진 적합식 $\hat{Y} = 15.3276 + 0.7803 X_1 - 0.0502 X_2$에서의 $X_1$의 계수가 3.5절에서 $X_2$의 계수에 대하여 묘사된 일련의 단순회귀식에 의하여 구해질 수 있음을 보여라.

In [3]:
performance = pd.read_csv("Supervisor_Performance.csv", sep="\t")

In [11]:
# y hat과 x_1의 회귀방정식 구하기
slope = get_slope(x=performance["X1 "], y=performance["Y "])
intercept = get_intercept(x=performance["X1 "], y=performance["Y "])
print(slope)
print(intercept)
y_hat = get_y_hat(slope=slope, intercept=intercept, x=performance["X1 "])
res_err_btw_y_and_x1 = [y-predicted_value for y, predicted_value in zip(performance["Y "], y_hat)]

0.754609818719365
14.37631940662363


In [12]:
# x1과 x2의 회귀 방정식 구하기
slope = get_slope(x=performance["X1 "], y=performance["X2 "])
intercept = get_intercept(x=performance["X1 "], y=performance["X2 "])
print(slope)
print(intercept)
y_hat = get_y_hat(slope=slope, intercept=intercept, x=performance["X1 "])
res_err_btw_x1_and_x2 = [y-predicted_value for y, predicted_value in zip(performance["X2 "], y_hat)]

0.5130319769703571
18.96540366710755


In [13]:
# y에 대한 x1의 잔차와 x2에 대한 x1의 잔차의 회귀방정 식 구하기
slope = get_slope(x=res_err_btw_x1_and_x2, y=res_err_btw_y_and_x1)
intercept = get_intercept(x=res_err_btw_x1_and_x2, y=res_err_btw_y_and_x1)
print(slope)
print(intercept)
y_hat = get_y_hat(slope=slope, intercept=intercept, x=res_err_btw_x1_and_x2)
res_err_btw_yx1_and_x2x1 = [y-predicted_value for y, predicted_value in zip(res_err_btw_y_and_x1, y_hat)]

-0.050159795205249866
-1.108431528942783e-14


#### 3.3
표 3.10은 통계학 과목을 수강한 22명의 학생들에 대하여 기말시험 성적 F와 두 번의 기초 시험 성적 P1, P2를 보여준다. 

In [14]:
examination = pd.read_csv("Examination_Data.csv", sep="\t")

In [16]:
#(a) 데이터에 다음의 모형들 각각을 적합하여라
exam_model_1 = smf.ols(formula="F ~ P1", data=examination)
result_1 = exam_model_1.fit()
result_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      F   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     81.14
Date:                Thu, 06 Aug 2020   Prob (F-statistic):           1.78e-08
Time:                        07:33:23   Log-Likelihood:                -65.930
No. Observations:                  22   AIC:                             135.9
Df Residuals:                      20   BIC:                             138.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -22.3424     11.564     -1.932      0.068     -46.464       1.780
P1             1.2605      0.140      9.008      0.000       0.969       1.552
==============================================================================
Omnibus:                        1.630   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.443   Jarque-Bera (JB):                1.284
Skew:                          -0.402   Prob(JB):                        0.526
Kurtosis:                       2.131   Cond. No.                         882.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
#(a) 데이터에 다음의 모형들 각각을 적합하여라
exam_model_2 = smf.ols(formula="F ~ P2", data=examination)
result_2 = exam_model_2.fit()
result_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      F   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                     122.9
Date:                Thu, 06 Aug 2020   Prob (F-statistic):           5.44e-10
Time:                        07:35:58   Log-Likelihood:                -62.128
No. Observations:                  22   AIC:                             128.3
Df Residuals:                      20   BIC:                             130.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8535      7.562     -0.245      0.809     -17.627      13.920
P2             1.0043      0.091     11.086      0.000       0.815       1.193
==============================================================================
Omnibus:                        2.902   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.234   Jarque-Bera (JB):                1.397
Skew:                          -0.558   Prob(JB):                        0.497
Kurtosis:                       3.529   Cond. No.                         693.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
#(a) 데이터에 다음의 모형들 각각을 적합하여라
exam_model_3 = smf.ols(formula="F ~ P1 + P2", data=examination)
result_3 = exam_model_3.fit()
result_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      F   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     74.07
Date:                Thu, 06 Aug 2020   Prob (F-statistic):           1.07e-09
Time:                        07:37:25   Log-Likelihood:                -59.840
No. Observations:                  22   AIC:                             125.7
Df Residuals:                      19   BIC:                             129.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -14.5005      9.236     -1.570      0.133     -33.831       4.830
P1             0.4883      0.233      2.096      0.050       0.001       0.976
P2             0.6720      0.179      3.748      0.001       0.297       1.047
==============================================================================
Omnibus:                        0.910   Durbin-Watson:                   1.778
Prob(Omnibus):                  0.634   Jarque-Bera (JB):                0.703
Skew:                          -0.410   Prob(JB):                        0.704
Kurtosis:                       2.696   Cond. No.                     1.29e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
#(b) 세 모형에 각각에 대하여 B0 = 0를 검정하여라

$F = B_0 + B_1P_1 + \epsilon$
- $B_0$의 p-value는 0.068로 95% 신뢰 수준 하에서 유의하지 않다.

$F = B_0 + B_2P_2 + \epsilon$
- $B_0$의 p-value는 0.809로 95% 신뢰 수준 하에서 유의하지 않다.

$F = B_0 + B_1P_1 + B_2P_2 + \epsilon$
- $B_0$의 p-value는 0.133로 95% 신뢰 수준 하에서 유의하지 않다.

In [22]:
#(c) 개별적으로 P1과 P2 중 어느 것이 F에 대하여 더 좋은 예측 변수 인가? P2의 기울기가 크고, p-value 또한 매우 작아 P2가 더 좋은 변수이다.

In [34]:
#(d) 첫 번째와 두 번째 기초 시험 성적이 78, 85점인 학생에 대하여 기말 시험 성적을 예측하기 위하여 어떤 모형을 사용하는 것이 좋은가?
# smf로 하는 법을 모르겠음;